In [1]:
import pandas as pd
from sqlalchemy import create_engine

import pymysql
pymysql.install_as_MySQLdb()

In [3]:
employment_file = "Resources/US districts.csv"
employment_df = pd.read_csv(employment_file)
employment_df.head()

State State Name Abbrev  115th Cong. Dist 2012 NAICS Code  \
0      1  Alabama       AL                 1          ------   
1      1  Alabama       AL                 2          ------   
2      1  Alabama       AL                 3          ------   
3      1  Alabama       AL                 4          ------   
4      1  Alabama       AL                 5          ------   

       NAICS Description Employment  Empl. - data flag Empl. - noise flag  \
0  Total for all sectors    236,720                NaN                  G   
1  Total for all sectors    215,224                NaN                  G   
2  Total for all sectors    179,100                NaN                  G   
3  Total for all sectors    188,796                NaN                  G   
4  Total for all sectors    255,797                NaN                  G   

   1st quarter payroll ($1,000)  1st quarter payroll - noise flag  \
0                      2,194,826                                G   
1                      1,951,501                                G   
2                      1,451,497                                G   
3                      1,550,243                                G   
4                      2,732,797                                G   

   Annual payroll ($1,000)  Annual payroll - noise flag  \
0                 9,449,314                           G   
1                 8,266,520                           G   
2                 6,150,074                           G   
3                 6,497,936                           G   
4                11,465,689                           G   

   Number of establishments   
0                     15,611  
1                     14,252  
2                     11,510  
3                     12,538  
4                     15,137

In [4]:
employment_df.columns.values

array(['State', 'State Name', 'Abbrev', '115th Cong. Dist',
       '2012 NAICS Code', 'NAICS Description', 'Employment',
       'Empl. - data flag', 'Empl. - noise flag',
       ' 1st quarter payroll ($1,000) ',
       '1st quarter payroll - noise flag', ' Annual payroll ($1,000) ',
       'Annual payroll - noise flag', ' Number of establishments '],
      dtype=object)

In [5]:
employment1_df = employment_df[['Abbrev', '115th Cong. Dist']].copy()
employment1_df.head()

Abbrev  115th Cong. Dist
0     AL                 1
1     AL                 2
2     AL                 3
3     AL                 4
4     AL                 5

In [6]:
employment1_df = employment1_df.rename(columns={"Abbrev":"state","115th Cong. Dist":"district","Employment":"employment", " Annual payroll ($1,000) ":"annual_payroll_in_1000", " Number of establishments ":"num_of_establishment"})
employment1_df.head()

state  district
0    AL         1
1    AL         2
2    AL         3
3    AL         4
4    AL         5

In [7]:
employment1_df.loc[employment1_df.district == 0, 'district'] = "00"
employment1_df.loc[employment1_df.district == 1, 'district'] = "01"
employment1_df.loc[employment1_df.district == 2, 'district'] = "02"
employment1_df.loc[employment1_df.district == 3, 'district'] = "03"
employment1_df.loc[employment1_df.district == 4, 'district'] = "04"
employment1_df.loc[employment1_df.district == 5, 'district'] = "05"
employment1_df.loc[employment1_df.district == 6, 'district'] = "06"
employment1_df.loc[employment1_df.district == 7, 'district'] = "07"
employment1_df.loc[employment1_df.district == 8, 'district'] = "08"
employment1_df.loc[employment1_df.district == 9, 'district'] = "09"


In [8]:
employment1_df.head()

state district
0    AL       01
1    AL       02
2    AL       03
3    AL       04
4    AL       05

In [9]:
state_dist_df = employment1_df['state'] + '_' +  employment1_df['district'].astype(str)
state_dist_df.head()

0    AL_01
1    AL_02
2    AL_03
3    AL_04
4    AL_05
dtype: object

In [10]:
employment1_df.insert(loc=0, column='state_dist', value=state_dist_df)
employment1_df.head()

state_dist state district
0      AL_01    AL       01
1      AL_02    AL       02
2      AL_03    AL       03
3      AL_04    AL       04
4      AL_05    AL       05

In [11]:
connection_string = "root:NewPassword@localhost:3306/election_prediction_db"
engine = create_engine(f'mysql://{connection_string}')

In [12]:
engine.table_names()

C:\Users\m_ron\Anaconda3\envs\PythonData\lib\site-packages\pymysql\cursors.py:170: Warning: (3719, "'utf8' is currently an alias for the character set UTF8MB3, which will be replaced by UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")
  result = self._query(query)


['candidate_finance',
 'district_demographic',
 'fips',
 'house_candidate',
 'us_congress',
 'us_congress_member',
 'us_congressional_district']

In [13]:
employment1_df.to_sql(name='us_congressional_district', con=engine, if_exists='append', index=False)

In [14]:
#confirm successful deposit into MySQL table
pd.read_sql_query('select * from us_congressional_district', con=engine).head()

id state_dist state district
0   1      AL_01    AL       01
1   2      AL_02    AL       02
2   3      AL_03    AL       03
3   4      AL_04    AL       04
4   5      AL_05    AL       05